In [ ]:
%pip install gurobipy
%pip install cvxpy
import gurobipy
import cvxpy as cp
import numpy as np
import pandas as pd
import csv

In [ ]:
# Load the csv
data = pd.read_csv('data.csv')

# Convert each column into a separate array
districts = data['DISTRICT'].tolist()
subdistricts = data['SCHEDULECODE'].tolist()
populations =  data['Population'].tolist()
rat_scores_og = data['RAT SCORE'].tolist()
rat_scores_normalized = data['100*RAT/POP SCORE (RAT/POP*100)'].tolist()
ped_scores_og = data['ROAD SCORE'].tolist()
ped_scores_normalized = data['ROAD/10 SCORE'].tolist()
am_scores = data['AM SCORE'].tolist()
pm_scores = data['PM SCORE'].tolist()

In [ ]:
timeslots = ["MA","MP", "WA","WP", "FA","FP", "TA","TP", "HA","HP", "SA","SP"]
timecombos = []
trucks = 431
truck_speed = 927 # people's trash per hour, associated with idealized traffic score of 1.

for a in ["MA","MP"]:
    for b in ["WA","WP"]:
        for c in ["FA","FP"]:
            timecombos.append((a,b,c))
for a in ["TA","TP"]:
    for b in ["HA","HP"]:
        for c in ["SA","SP"]:
            timecombos.append((a,b,c))

sub_ct = len(subdistricts)
ts_ct = len(timeslots)
com_ct = len(timecombos)

In [ ]:
def negative_impact(pop, rats, road, time):
    if time == 'AM':
        return rats + road # 1 and 1 baseline
    else:
        return 1.3 * rats + 2 * road # 1.3 and 2 baseline

def time_cost(pop, traffic, ts):
    base = pop * traffic / truck_speed
    if ts < 10:
        return base
    else: return 0.9 * base # Saturday
    

In [ ]:
tc_arr = np.zeros((sub_ct, ts_ct))
for i in range(sub_ct):
    for j in range(ts_ct):
        if j%2 == 0:
            tc_arr[i][j] = time_cost(populations[i], am_scores[i], j)
        else:
            tc_arr[i][j] = time_cost(populations[i], pm_scores[i], j)

ni_arr = np.zeros((sub_ct, ts_ct))
for i in range(sub_ct):
    for j in range(0, ts_ct, 2): # ideally we'd know pedestrian traffic per time slot
        ni_arr[i][j] = negative_impact(populations[i], rat_scores_normalized[i], ped_scores_normalized[i], 'AM')
        ni_arr[i][j + 1] = negative_impact(populations[i], rat_scores_normalized[i], ped_scores_normalized[i], 'PM')

In [ ]:
# Dict representing time costs for subdistrict i with collection at time slot j
time_costs_slot = {}
for i in range(sub_ct):
    for j in range(ts_ct):
        time_costs_slot[i, timeslots[j]] = tc_arr[i][j]

# Dict representing negative impact for subdistrict i with collection at time slot j
negative_impact_slot = {}
for i in range(sub_ct):
    for j in range(ts_ct):
        negative_impact_slot[i, timeslots[j]] = ni_arr[i][j]

# Array representing negative impact for subdistrict i with time combo j
negative_impact_combo = np.zeros((sub_ct, 16))
for i in range(sub_ct):
    for j in range(len(timecombos)):
        total = 0
        for time_slot in timeslots:
            total += (time_slot in timecombos[j]) * negative_impact_slot[i, time_slot]
        negative_impact_combo[i][j] = total


In [ ]:
def solve_problem(trucks):
    avs = {}
    for subdistrict in subdistricts:
        for time_combo in timecombos:
            avs[subdistrict, time_combo] = cp.Variable(boolean = True)

    # Define objective
    neg_imp_total = sum(negative_impact_combo[i][j] * avs[subdistricts[i],timecombos[j]] for i in range(sub_ct) for j in range(com_ct))

    # Create constraints
    constraints = []
    for timeslot in timeslots:
        constraints += [ sum(time_costs_slot[i, timeslot] * (timeslot in timecombos[j]) * avs[subdistricts[i], timecombos[j]] for i in range(sub_ct) for j in range(com_ct)) <= trucks]
    for subdistrict in subdistricts:
        constraints += [ sum(avs[subdistrict, time_combo] for time_combo in timecombos) == 1]

      # Solve problem
    prob = cp.Problem(cp.Minimize(neg_imp_total), constraints)
    prob.solve(solver = cp.GUROBI)

    assignments = {}
    for subdistrict in subdistricts:
        for time_combo in timecombos:
            if avs[subdistrict, time_combo].value is None:
                break
            elif avs[subdistrict, time_combo].value > 0.1: # this fixed a close-to-one but not quite error
                assignments[subdistrict] = time_combo
                
    return (prob.value, assignments)

In [ ]:
obj, assign = solve_problem(431)
print(obj)

In [ ]:
output_file = "output.csv"

# Write the assignment dictionary to a csv
with open(output_file, mode="w", newline="") as file:
    writer = csv.writer(file)
    
    # Header
    writer.writerow(["Section", "Schedule"])
    
    # Write each key and its corresponding values
    for key, values in assign.items():
        writer.writerow([key, values])


In [ ]:
# This outputs the average negative impact of randomly assigning timeslots (and with no accounting for feasibility).
# ~ 2578 or thereabouts, significantly higher than the optimized feasible solution with the lowest number of trucks

rng = np.random.default_rng()

grand_total = 0

# randomly assign time slots
for k in range(1000):
    total = 0
    for i in range(sub_ct):
        total += negative_impact_combo[i][np.random.randint(0, 16)]
    grand_total += total

print(grand_total/1000)


In [ ]:
def solve_district_problem(sub_indices, trucks): #list the indices of subdistricts in your district(s), trucks in garage
    avs = {}
    for i in sub_indices:
        for time_combo in timecombos:
            avs[subdistricts[i], time_combo] = cp.Variable(boolean = True)

    # Define objective
    neg_imp_total = sum(negative_impact_combo[i][j] * avs[subdistricts[i],timecombos[j]] for i in sub_indices for j in range(com_ct))

    # Create constraints
    constraints = []
    for timeslot in timeslots:
        constraints += [ sum(time_costs_slot[i, timeslot] * (timeslot in timecombos[j]) * avs[subdistricts[i], timecombos[j]] for i in sub_indices for j in range(com_ct)) <= trucks]
    for i in sub_indices:
        constraints += [ sum(avs[subdistricts[i], time_combo] for time_combo in timecombos) == 1]

      # Solve problem
    prob = cp.Problem(cp.Minimize(neg_imp_total), constraints)
    prob.solve(solver = cp.GUROBI)

    assignments = {}
    for i in sub_indices:
        for time_combo in timecombos:
            if avs[subdistricts[i], time_combo].value is None:
                break
            elif avs[subdistricts[i], time_combo].value > 0.1:
                assignments[subdistricts[i]] = time_combo
    return (prob.value, assignments)

In [ ]:
# the next two cells should compute ideal allocations without sharing (except for between districts that share garages)
# however, some of these values result in no feasible solutions, for reasons discussed in the paper

truck_dist = [21, 24, 42, 34, 16, 40, 57, 57, 29, 34, 32, 46]
sub_indices_dist_alpha = [4, 6, 8, 6, 4, 6, 10, 10, 6, 6, 6, 8]
sid = []
incumbent = 0
for i in sub_indices_dist_alpha:
    sid.append(range(incumbent, incumbent + i))
    incumbent +=i

truck_garages = [21 + 24 + 16, 42 + 40, 34 + 57, 57, 29, 34, 32, 46]
sid_garages = [list(sid[0]) + list(sid[1]) + list(sid[4]),
                      list(sid[2]) + list(sid[5]), list(sid[3]) + list(sid[6]),
                      sid[7], sid[8], sid[9], sid[10], sid[11]]

In [ ]:
objectives = []
assignments = {}
for i in range(len(sid_garages)):
    obj, assign = solve_district_problem(sid_garages[i], truck_garages[i])
    print(obj)
    print(assign)
    if assign:
        for j in (sid_garages[i]):
            assignments[subdistricts[j]] = assign[subdistricts[j]]
    else:
        print("garage", i, "is infeasible")